# Sentinel-1 SLC Burst Search Examples

This notebook demonstrates how to search for Sentinel-1 SLC burst products using the phidown library's burst mode feature.

> **Note:** If you encounter errors about missing `burst_mode` parameter, please restart the kernel (**Kernel > Restart Kernel**) and run the cells again. This ensures the latest phidown version with burst mode support is loaded.

## What are Sentinel-1 Bursts?

Sentinel-1 operates in Interferometric Wide (IW) and Extra-Wide (EW) modes using a ScanSAR technique called TOPSAR (Terrain Observation with Progressive Scans SAR).
In these modes, the radar antenna repeatedly switches between several adjacent sub-swaths, collecting short sequences of radar pulses during each observation.
Each of these short pulse sequences is known as a burst, which represents the smallest imaging unit in Sentinel-1 SLC data.

A standard IW or EW SLC product combines data from many bursts.
Each sub-swath is processed into an image containing a series of slightly overlapping bursts, and all sub-swaths (three for IW, five for EW) are then merged to form the final SLC product. Although the product contains multiple bursts, each burst can also be processed and used individually.

## Features:
- Search for individual Sentinel-1 SLC bursts (available from August 2, 2024 onwards)
- Filter by burst-specific parameters (Burst ID, Swath, Polarization, etc.)
- Combine with standard filters (temporal, spatial, orbit direction)
- Access bursts from parent products without downloading the full scene

## Key Burst Parameters:
- **burst_id**: Unique identifier for a specific burst
- **absolute_burst_id**: Global unique burst identifier
- **swath_identifier**: Swath name (IW1, IW2, IW3 for Interferometric Wide, EW1-EW5 for Extra Wide)
- **parent_product_name**: Name of the parent SLC product
- **parent_product_type**: Type of parent product (IW_SLC__1S or EW_SLC__1S)
- **operational_mode**: Acquisition mode (IW or EW)
- **polarisation_channels**: Polarization (VV, VH, HH, HV)
- **platform_serial_identifier**: Sentinel-1 satellite (A or B)
- **relative_orbit_number**: Relative orbit number
- **datatake_id**: Datatake identifier

## Usage:
1. Set `burst_mode=True` in the query_by_filter method
2. Add burst-specific filters as needed
3. Execute the query to retrieve burst results

### Key Takeaways:

1. **Enable Burst Mode**: Always set `burst_mode=True` to search for individual bursts instead of full products
2. **Data Availability**: Burst data is available from August 2, 2024 onwards
3. **Combine Filters**: You can combine temporal, spatial, and burst-specific filters for targeted searches
4. **Swath Selection**: Use `swath_identifier` to select specific swaths (IW1, IW2, IW3 for IW mode)
5. **Polarization**: Filter by polarization channels (VV, VH, HH, HV) based on your application
6. **Orbit Parameters**: Use `orbit_direction` and `relative_orbit_number` for consistent time-series
7. **Parent Products**: Search bursts from specific parent products using `parent_product_name`

### Common Use Cases:

- **InSAR Time Series**: Search for specific burst IDs across multiple dates with consistent orbit parameters
- **Regional Analysis**: Use AOI + swath/polarization filters to get bursts covering your study area
- **Product Decomposition**: Retrieve all bursts from a parent SLC product for individual processing
- **Systematic Monitoring**: Filter by relative orbit number and burst ID for regular acquisitions

### Valid Parameter Values:

- **swath_identifier**: 'IW1', 'IW2', 'IW3', 'EW1', 'EW2', 'EW3', 'EW4', 'EW5'
- **parent_product_type**: 'IW_SLC__1S', 'EW_SLC__1S'
- **operational_mode**: 'IW', 'EW'
- **polarisation_channels**: 'VV', 'VH', 'HH', 'HV'
- **platform_serial_identifier**: 'A' (Sentinel-1A), 'B' (Sentinel-1B)
- **orbit_direction**: 'ASCENDING', 'DESCENDING'

For more information, visit the [Copernicus Data Space Ecosystem documentation](https://dataspace.copernicus.eu/)

In [ ]:
# Auto-reload modules during development
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Import Required Libraries

Import the CopernicusDataSearcher class from phidown for burst searching.

Note: If you get an error about 'burst_mode' parameter not found, please:
1. Restart the kernel (Kernel > Restart Kernel)
2. Re-run this cell
This ensures the latest phidown version with burst mode support is loaded.
"""
from phidown.search import CopernicusDataSearcher
import pandas as pd

# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Verify burst mode is available
print('Checking if burst mode is available...')
import inspect
sig = inspect.signature(CopernicusDataSearcher.query_by_filter)
if 'burst_mode' in sig.parameters:
    print('✓ Burst mode is available!')
else:
    print('✗ Burst mode not found. Please restart the kernel and re-run this cell.')

## Example 1: Basic Burst Search with Temporal Filter

The simplest burst search uses only temporal filters. Set `burst_mode=True` to search the Bursts endpoint instead of the Products endpoint.

In [ ]:
"""
Basic Burst Search

Search for Sentinel-1 SLC bursts within a specific time range.
Data is available from August 2, 2024 onwards.
"""
searcher = CopernicusDataSearcher()

# Configure search for bursts with temporal filter
searcher.query_by_filter(
    burst_mode=True,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

# Execute the query
df = searcher.execute_query()
print(f'Number of results: {len(df)}')
print(f'Total available results: {searcher.num_results}')

# Display first few results
searcher.display_results(top_n=5)

# Example 1B: Download API for Burst

To access a specific burst, the only required parameter is the burst ID. In this case, the CDSE API operates differently from standard product retrieval, as burst products are generated on demand rather than retrieved from pre-stored files. Consequently, S3 commands are not applicable. Instead, a POST request is sent to the CDSE service, which dynamically produces and returns the requested burst to the user. 

In this case, a different type of credential is required—an access token—which can be obtained using the standard CDSE user credentials.

[Implementation based on: https://github.com/eu-cdse/notebook-samples/blob/main/geo/bursts_processing_on_demand.ipynb. All credits to CDSE]

In [ ]:
from pathlib import Path
from phidown.downloader import download_burst_on_demand, get_token

In [ ]:
access_token = get_token(username='your_cdse_username',
                        password='your_cdse_password')

In [ ]:
burst_id = df.iloc[0]['Id']

download_burst_on_demand(burst_id=burst_id, token=access_token,
              output_dir=Path('./'))

## Example 2: Burst Search with Spatial Filter (Area of Interest)

You can combine burst mode with spatial filtering using a WKT polygon to define your area of interest.

In [ ]:
"""
Burst Search with AOI

Search for bursts that intersect with a specific geographic area.
The AOI is defined as a WKT (Well-Known Text) polygon.
"""
# Define an area of interest (example: region in central Europe)
aoi_wkt = """POLYGON((12.655118166047592 47.44667197521409, 
                       21.39065656328509 48.347694733853245, 
                       28.334291357162826 41.877123516783655, 
                       17.47086198383573 40.35854475076158, 
                       12.655118166047592 47.44667197521409))"""

searcher = CopernicusDataSearcher()

# Configure search with AOI and temporal filter
searcher.query_by_filter(
    burst_mode=True,
    aoi_wkt=aoi_wkt,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display results
df = searcher.execute_query()
print(f'Found {len(df)} bursts in the specified area')
searcher.display_results(top_n=5)

## Example 3: Search by Specific Burst ID

If you know the Burst ID you're looking for, you can search for it directly.

In [ ]:
"""
Search by Burst ID

Retrieve all acquisitions of a specific burst by its ID.
Burst IDs are integers that uniquely identify a burst location.
"""
searcher = CopernicusDataSearcher()

# Search for a specific burst ID
searcher.query_by_filter(
    burst_mode=True,
    burst_id=15804,  # Example burst ID
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} acquisitions of burst {15804}')
searcher.display_results(top_n=5)

## Example 4: Search by Swath Identifier and Polarization

Filter bursts by specific swath (IW1, IW2, IW3) and polarization channel (VV, VH, HH, HV).

In [ ]:
"""
Filter by Swath and Polarization

Search for bursts from a specific swath with specific polarization.
Swath identifiers: IW1, IW2, IW3 (Interferometric Wide) or EW1-EW5 (Extra Wide)
Polarization channels: VV, VH, HH, HV
"""
searcher = CopernicusDataSearcher()

# Search for IW2 swath with VV polarization
searcher.query_by_filter(
    burst_mode=True,
    swath_identifier='IW2',
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=10,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts from swath IW2 with VV polarization')
searcher.display_results(top_n=5)

## Example 5: Search Bursts from a Specific Parent Product

If you know the parent SLC product name, you can retrieve all its constituent bursts.

In [ ]:
"""
Search by Parent Product Name

Retrieve all bursts from a specific parent SLC product.
This is useful when you want to access individual bursts from a known product.
"""
searcher = CopernicusDataSearcher()

# Example parent product name (replace with an actual product name from your results)
parent_product = 'S1A_IW_SLC__1SDV_20240802T060719_20240802T060746_055030_06B44E_E7CC.SAFE'

searcher.query_by_filter(
    burst_mode=True,
    parent_product_name=parent_product,
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    polarisation_channels='VV',
    top=1000  # Get all bursts from this product
)


# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts in parent product')
print(f'Parent product: {parent_product}\n')
searcher.display_results(top_n=100)

## Example 6: Filter by Orbit Direction and Relative Orbit Number

Combine orbit parameters with burst search to find specific orbital acquisitions.

In [ ]:
"""
Filter by Orbit Parameters

Search for bursts from a specific orbit direction and relative orbit number.
This is useful for consistent time-series analysis.
"""
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    orbit_direction='DESCENDING',
    relative_orbit_number=8,
    operational_mode='IW',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-10T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts from descending orbit #8')
searcher.display_results(top_n=5)

## Example 7: Advanced Multi-Parameter Burst Search

Combine multiple burst-specific parameters for highly targeted searches.

In [ ]:
"""
Advanced Multi-Parameter Search

Demonstrate combining multiple burst-specific filters for precise queries.
This example searches for a specific burst ID with additional constraints.
"""
searcher = CopernicusDataSearcher()

searcher.query_by_filter(
    burst_mode=True,
    burst_id=15804,
    swath_identifier='IW2',
    parent_product_type='IW_SLC__1S',
    orbit_direction='DESCENDING',
    relative_orbit_number=8,
    operational_mode='IW',
    polarisation_channels='VV',
    platform_serial_identifier='A',  # Sentinel-1A
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-15T00:00:00',
    top=20,
    count=True
)

print(f'Query URL: {searcher._build_query()}\n')

# Execute and display
df = searcher.execute_query()
print(f'Found {len(df)} bursts matching all criteria')
print(f'Total available: {searcher.num_results}')
searcher.display_results(top_n=10)

## Example 8: Analyzing Burst Results

Once you have burst results, you can analyze the data to understand burst distribution, coverage, and other characteristics.

In [ ]:
"""
Analyze Burst Search Results

Perform basic analysis on burst search results to understand:
- Distribution across swaths
- Polarization channels available
- Parent products
- Temporal coverage
"""
searcher = CopernicusDataSearcher()

# Search for bursts with broad criteria
searcher.query_by_filter(
    burst_mode=True,
    orbit_direction='DESCENDING',
    swath_identifier='IW2',
    polarisation_channels='VV',
    start_date='2024-08-01T00:00:00',
    end_date='2024-08-03T00:00:00',
    top=100,
    count=True
)

# Execute query
df = searcher.execute_query()

print(f'=== Burst Search Results Analysis ===\n')
print(f'Total bursts found: {len(df)}')
print(f'Total available: {searcher.num_results}\n')

# Analyze burst distribution
if len(df) > 0:
    print(f'=== Burst Characteristics ===')
    
    # Check available columns
    if 'SwathIdentifier' in df.columns:
        print(f'\nSwath Distribution:')
        print(df['SwathIdentifier'].value_counts())
    
    if 'PolarisationChannels' in df.columns:
        print(f'\nPolarization Channels:')
        print(df['PolarisationChannels'].value_counts())
    
    if 'ParentProductName' in df.columns:
        print(f'\nNumber of unique parent products: {df["ParentProductName"].nunique()}')
    
    if 'BurstId' in df.columns:
        print(f'\nNumber of unique burst IDs: {df["BurstId"].nunique()}')
        print(f'Burst ID range: {df["BurstId"].min()} - {df["BurstId"].max()}')
    
    print(f'\n=== Sample Results ===')
    searcher.display_results(top_n=5)